# Metrics

## Example is given @ the end of this file

In [6]:
"""
author: Clément Zotti (clement.zotti@usherbrooke.ca)
date: April 2017

DESCRIPTION :
to generate metrics for two images:
    - metrics()

And it is callable from the command line (see below).
Each function provided in this script has comments to understand
how they works.

HOW-TO:

The first option will print in the console the dice and volume of each class for the given image.
The second option wiil ouput a csv file where each images will have the dice and volume of each class.

Link: http://acdc.creatis.insa-lyon.fr

"""

import os
from glob import glob
import time
import re
import argparse
import nibabel as nib
import pandas as pd
from medpy.metric.binary import hd, dc     # !pip install medpy <-- make sure medpy is installed
import numpy as np



HEADER = ["Name", "Dice LV", "Volume LV", "Err LV(ml)",
          "Dice RV", "Volume RV", "Err RV(ml)",
          "Dice MYO", "Volume MYO", "Err MYO(ml)"]

#
# Functions to process files, directories and metrics
#
def metrics(img_gt, img_pred, voxel_size):
    """
    Function to compute the metrics between two segmentation maps given as input.
    ----------
    
    Abbreviations:
    ----------
    LV: left ventricle
    RV: right ventricle
    MYO: myocardium
    
    Parameters
    ----------
    img_gt: np.array
    Array of the ground truth segmentation map.  (LV --> class 3, RV --> class 1, myocardium --> class 2)

    img_pred: np.array
    Array of the predicted segmentation map.

    voxel_size: list, tuple or np.array
    The size of a voxel of the images used to compute the volumes (in mm).
    
    For example: for voxels of size 2mm by  1mm by 8mm, use voxel_size = np.array([2 1 8])

    Return
    ------
    A list of metrics in this order, [Dice LV, Volume LV, Err LV(ml),
    Dice RV, Volume RV, Err RV(ml), Dice MYO, Volume MYO, Err MYO(ml)]
    """

    if img_gt.ndim != img_pred.ndim:
        raise ValueError("The arrays 'img_gt' and 'img_pred' should have the "
                         "same dimension, {} against {}".format(img_gt.ndim,
                                                                img_pred.ndim))

    res = []
    # Loop on each classes of the input images
    for c in [3, 1, 2]:
        # Copy the gt image to not alterate the input
        gt_c_i = np.copy(img_gt)
        gt_c_i[gt_c_i != c] = 0

        # Copy the pred image to not alterate the input
        pred_c_i = np.copy(img_pred)
        pred_c_i[pred_c_i != c] = 0

        # Clip the value to compute the volumes
        gt_c_i = np.clip(gt_c_i, 0, 1)
        pred_c_i = np.clip(pred_c_i, 0, 1)

        # Compute the Dice
        dice = dc(gt_c_i, pred_c_i)

        # Compute volume
        volpred = pred_c_i.sum() * np.prod(voxel_size) / 1000.
        volgt = gt_c_i.sum() * np.prod(voxel_size) / 1000.

        res += [dice, volpred, volpred-volgt]

    return res




# maybe usefull later!!! DO NOT DELETE!!!
##################################################################################################
##################################################################################################
##################################################################################################

# def compute_metrics_on_files(path_gt, path_pred):
#     """
#     Function to give the metrics for two files

#     Parameters
#     ----------

#     path_gt: string
#     Path of the ground truth image.

#     path_pred: string
#     Path of the predicted image.
#     """
#     gt, _, header = load_nii(path_gt)
#     pred, _, _ = load_nii(path_pred)
#     zooms = header.get_zooms()

#     name = os.path.basename(path_gt)
#     name = name.split('.')[0]
#     res = metrics(gt, pred, zooms)
#     res = ["{:.3f}".format(r) for r in res]

#     formatting = "{:>14}, {:>7}, {:>9}, {:>10}, {:>7}, {:>9}, {:>10}, {:>8}, {:>10}, {:>11}"
#     print(formatting.format(*HEADER))
#     print(formatting.format(name, *res))


# def compute_metrics_on_directories(dir_gt, dir_pred):
#     """
#     Function to generate a csv file for each images of two directories.

#     Parameters
#     ----------

#     path_gt: string
#     Directory of the ground truth segmentation maps.

#     path_pred: string
#     Directory of the predicted segmentation maps.
#     """
#     lst_gt = sorted(glob(os.path.join(dir_gt, '*')), key=natural_order)
#     lst_pred = sorted(glob(os.path.join(dir_pred, '*')), key=natural_order)

#     res = []
#     for p_gt, p_pred in zip(lst_gt, lst_pred):
#         if os.path.basename(p_gt) != os.path.basename(p_pred):
#             raise ValueError("The two files don't have the same name"
#                              " {}, {}.".format(os.path.basename(p_gt),
#                                                os.path.basename(p_pred)))

#         gt, _, header = load_nii(p_gt)
#         pred, _, _ = load_nii(p_pred)
#         zooms = header.get_zooms()
#         res.append(metrics(gt, pred, zooms))

#     lst_name_gt = [os.path.basename(gt).split(".")[0] for gt in lst_gt]
#     res = [[n,] + r for r, n in zip(res, lst_name_gt)]
#     df = pd.DataFrame(res, columns=HEADER)
#     df.to_csv("results_{}.csv".format(time.strftime("%Y%m%d_%H%M%S")), index=False)

# def main(path_gt, path_pred):
#     """
#     Main function to select which method to apply on the input parameters.
#     """
#     if os.path.isfile(path_gt) and os.path.isfile(path_pred):
#         compute_metrics_on_files(path_gt, path_pred)
#     elif os.path.isdir(path_gt) and os.path.isdir(path_pred):
#         compute_metrics_on_directories(path_gt, path_pred)
#     else:
#         raise ValueError(
#             "The paths given needs to be two directories or two files.")


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser(
#         description="Script to compute ACDC challenge metrics.")
#     parser.add_argument("GT_IMG", type=str, help="Ground Truth image")
#     parser.add_argument("PRED_IMG", type=str, help="Predicted image")
#     args = parser.parse_args()
#     main(args.GT_IMG, args.PRED_IMG)

##################################################################################################
##################################################################################################
##################################################################################################

usage: ipykernel_launcher.py [-h] GT_IMG PRED_IMG
ipykernel_launcher.py: error: the following arguments are required: PRED_IMG


SystemExit: 2

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Example

In [17]:
ground_truth = np.array([2,2,2,1,1,0,0,0,1,1,3,3,3,3]) #(LV --> class 3, RV --> class 1, myocardium --> class 2)
prediction =   np.array([2,2,1,1,1,1,0,0,0,1,3,3,3,0]) # prediction from neural network 
voxel_size = np.array([10,10,10])

In [18]:
res = metrics(ground_truth, prediction, voxel_size) # use function defined above to determine the metrics
res = ["{:.3f}".format(r) for r in res]

formatting = "{:>14}, {:>7}, {:>9}, {:>10}, {:>7}, {:>9}, {:>10}, {:>8}, {:>10}, {:>11}"
print(formatting.format(*HEADER))
print(formatting.format('patiënt 1', *res))

          Name, Dice LV, Volume LV, Err LV(ml), Dice RV, Volume RV, Err RV(ml), Dice MYO, Volume MYO, Err MYO(ml)
     patiënt 1,   0.857,     3.000,     -1.000,   0.667,     5.000,      1.000,    0.800,      2.000,      -1.000
